In [ ]:
from keybert import KeyBERT
import re
import math
import json
import requests
# from paddlenlp import Taskflow
# seg_accurate = Taskflow("word_segmentation", mode="accurate", user_dict="user_dict.txt")

# import spacy
# zh_model = spacy.load("zh_core_web_lg")
# key_model = KeyBERT(model=zh_model)


from transformers.pipelines import pipeline
hf_model = pipeline("feature-extraction", model="ernie-gram-zh")
key_model = KeyBERT(model=hf_model)


# from sentence_transformers import SentenceTransformer
# sentence_model = SentenceTransformer("xlm-r-distilroberta-base-paraphrase-v1")
# key_model = KeyBERT(model=sentence_model)

In [ ]:

stoplist =list(set([i.strip() for i in open(r'stopwords.txt',encoding='utf-8').readlines()]))

def preprocess(document):
    document = re.sub(u"([^\u4e00-\u9fa5-z0-9，。？！“”：（）.])","",document)
    return document

def cut(text):
    
    obj = {"str": text}
    req_str = json.dumps(obj).encode()

    url = "https://texsmart.qq.com/api"
    r = requests.post(url, data=req_str)
    r.encoding = "utf-8"
    result=json.loads(r.text)
    cut=[i['str'] for i in result['phrase_list'] ]
    tag=[i['tag'] for i in result['phrase_list']]
    results = dict(zip(cut,tag))
    return results

def keyword(doc):
    postag=['NN','NR']
    try:
        docs_cut=cut(preprocess(doc))
        doc_cut=' '.join(docs_cut.keys())
#         print(len(doc_cut))
        top_n=math.ceil(len(doc_cut)/20)
        if top_n>5:top_n=5
        keywords = key_model.extract_keywords(doc_cut, keyphrase_ngram_range=(1, 1),stop_words=stoplist,top_n=top_n,use_mmr=True, diversity=0.7)
        keywords=[i for i in keywords if i[1]>0.5]
        results=[__ for __ in keywords if docs_cut[__[0]] in postag]

        return results
    except:
        return ''



In [ ]:
doc='''习近平总书记指出，发展数字经济是把握新一轮科技革命和产业变革新机遇的战略选择，是新一轮国际竞争重点领域，我们一定要抓住先机，抢占未来发展制高点。党的十八大以来，我国深入实施数字经济发展战略，数字经济成为我国经济发展中创新最活跃、增长速度最快、影响最广泛的领域，推动生产生活方式发生深刻变革。
'''
keyword(doc)

In [ ]:
import gradio as gr

def sentiment_analysis(text):
    result=keyword(text)
    results={}
    for i in result:
        results[i[0]]=i[1]
    return results

title = "Keyword identification"

examples = [
    ["习近平总书记指出，发展数字经济是把握新一轮科技革命和产业变革新机遇的战略选择，是新一轮国际竞争重点领域，我们一定要抓住先机，抢占未来发展制高点。党的十八大以来，我国深入实施数字经济发展战略，数字经济成为我国经济发展中创新最活跃、增长速度最快、影响最广泛的领域，推动生产生活方式发生深刻变革。"],
    ["试试千字文,南北朝 · 周兴嗣 天地玄黄，宇宙洪荒。 日月盈昃，辰宿列张。 寒来暑往，秋收冬藏。 闰余成岁，律吕调阳。 云腾致雨，露结为霜。 金生丽水，玉出昆冈。 剑号巨阙，珠称夜光。 果珍李柰，菜重芥姜。 海咸河淡，鳞潜羽翔。 龙师火帝，鸟官人皇。 始制文字，乃服衣裳。 推位让国，有虞陶唐。 吊民伐罪，周发殷汤。 坐朝问道，垂拱平章。 爱育黎首，臣伏戎羌。 遐迩一体，率宾归王。 鸣凤在竹，白驹食场。 化被草木，赖及万方。 盖此身发，四大五常。 恭惟鞠养，岂敢毁伤。 女慕贞洁，男效才良。 知过必改，得能莫忘。 罔谈彼短，靡恃己长。 信使可覆，器欲难量。 墨悲丝染，诗赞羔羊。 景行维贤，克念作圣。 德建名立，形端表正。 空谷传声，虚堂习听。 祸因恶积，福缘善庆。 尺璧非宝，寸阴是竞。 资父事君，曰严与敬。 孝当竭力，忠则尽命。 临深履薄，夙兴温凊。 似兰斯馨，如松之盛 。 川流不息，渊澄取映。 容止若思，言辞安定。 笃初诚美，慎终宜令。 荣业所基，籍甚无竟。 学优登仕，摄职从政。 存以甘棠，去而益咏。 乐殊贵贱，礼别尊卑。 上和下睦，夫唱妇随。 外受傅训，入奉母仪。 诸姑伯叔，犹子比儿。 孔怀兄弟，同气连枝。 交友投分，切磨箴规。 仁慈隐恻，造次弗离。 节义廉退，颠沛匪亏。 性静情逸，心动神疲。 守真志满，逐物意移。 坚持雅操，好爵自縻。 都邑华夏，东西二京。 背邙面洛，浮渭据泾。 宫殿盘郁，楼观飞惊。 图写禽兽，画彩仙灵。 丙舍旁启，甲帐对楹。 肆筵设席，鼓瑟吹笙。 升阶纳陛，弁转疑星。 右通广内，左达承明。 既集坟典，亦聚群英。 杜稿钟隶，漆书壁经。 府罗将相，路侠槐卿。 户封八县，家给千兵。 高冠陪辇，驱毂振缨。 世禄侈富，车驾肥轻。 策功茂实，勒碑刻铭。 盘溪伊尹，佐时阿衡。 奄宅曲阜，微旦孰营。 桓公匡合，济弱扶倾。 绮回汉惠，说感武丁。 俊义密勿，多士实宁。 晋楚更霸，赵魏困横。 假途灭虢，践土会盟。 何遵约法，韩弊烦刑。 起翦颇牧，用军最精。 宣威沙漠，驰誉丹青。 九州禹迹，百郡秦并。 岳宗泰岱，禅主云亭。 雁门紫塞，鸡田赤诚。 昆池碣石，钜野洞庭。 旷远绵邈，岩岫杳冥。 治本于农，务兹稼穑。 俶载南亩，我艺黍稷。 税熟贡新，劝赏黜陟。 孟轲敦素，史鱼秉直。 庶几中庸，劳谦谨敕。 聆音察理，鉴貌辨色。 贻厥嘉猷，勉其祗植。 省躬讥诫，宠增抗极。 殆辱近耻，林皋幸即。 两疏见机，解组谁逼。 索居闲处，沉默寂寥。 求古寻论，散虑逍遥。 欣奏累遣，戚谢欢招。 渠荷的历，园莽抽条。 枇杷晚翠，梧桐蚤凋。 陈根委翳，落叶飘摇。 游鹍独运，凌摩绛霄。 耽读玩市，寓目囊箱。 易輶攸畏，属耳垣墙。 具膳餐饭，适口充肠。 饱饫烹宰，饥厌糟糠。 亲戚故旧，老少异粮。 妾御绩纺，侍巾帷房。 纨扇圆洁，银烛炜煌。 昼眠夕寐，蓝笋象床。 弦歌酒宴，接杯举殇。 矫手顿足，悦豫且康。 嫡后嗣续，祭祀烝尝。 稽颡再拜，悚惧恐惶。 笺牒简要，顾答审详。 骸垢想浴，执热愿凉。 驴骡犊特，骇跃超骧。 诛斩贼盗，捕获叛亡。 布射僚丸，嵇琴阮箫。 恬笔伦纸，钧巧任钓。 释纷利俗，并皆佳妙。 毛施淑姿，工颦妍笑。 年矢每催，曦晖朗曜。 璇玑悬斡，晦魄环照。 指薪修祜，永绥吉劭。 矩步引领，俯仰廊庙。 束带矜庄，徘徊瞻眺。 孤陋寡闻，愚蒙等诮。 谓语助者，焉哉乎也。,试试千字文 南北朝 · 周兴嗣 天地玄黄，宇宙洪荒。 日月盈昃，辰宿列张。 寒来暑往，秋收冬藏。 闰余成岁，律吕调阳。 云腾致雨，露结为霜。 金生丽水，玉出昆冈。 剑号巨阙，珠称夜光。 果珍李柰，菜重芥姜。 海咸河淡，鳞潜羽翔。 龙师火帝，鸟官人皇。 始制文字，乃服衣裳。 推位让国，有虞陶唐。 吊民伐罪，周发殷汤。 坐朝问道，垂拱平章。 爱育黎首，臣伏戎羌。 遐迩一体，率宾归王。 鸣凤在竹，白驹食场。 化被草木，赖及万方。 盖此身发，四大五常。 恭惟鞠养，岂敢毁伤。 女慕贞洁，男效才良。 知过必改，得能莫忘。 罔谈彼短，靡恃己长。 信使可覆，器欲难量。 墨悲丝染，诗赞羔羊。 景行维贤，克念作圣。 德建名立，形端表正。 空谷传声，虚堂习听。 祸因恶积，福缘善庆。 尺璧非宝，寸阴是竞。 资父事君，曰严与敬。 孝当竭力，忠则尽命。 临深履薄，夙兴温凊。 似兰斯馨，如松之盛 。 川流不息，渊澄取映。 容止若思，言辞安定。 笃初诚美，慎终宜令。 荣业所基，籍甚无竟。 学优登仕，摄职从政。 存以甘棠，去而益咏。 乐殊贵贱，礼别尊卑。 上和下睦，夫唱妇随。 外受傅训，入奉母仪。 诸姑伯叔，犹子比儿。 孔怀兄弟，同气连枝。 交友投分，切磨箴规。 仁慈隐恻，造次弗离。 节义廉退，颠沛匪亏。 性静情逸，心动神疲。 守真志满，逐物意移。 坚持雅操，好爵自縻。 都邑华夏，东西二京。 背邙面洛，浮渭据泾。 宫殿盘郁，楼观飞惊。 图写禽兽，画彩仙灵。 丙舍旁启，甲帐对楹。 肆筵设席，鼓瑟吹笙。 升阶纳陛，弁转疑星。 右通广内，左达承明。 既集坟典，亦聚群英。 杜稿钟隶，漆书壁经。 府罗将相，路侠槐卿。 户封八县，家给千兵。 高冠陪辇，驱毂振缨。 世禄侈富，车驾肥轻。 策功茂实，勒碑刻铭。 盘溪伊尹，佐时阿衡。 奄宅曲阜，微旦孰营。 桓公匡合，济弱扶倾。 绮回汉惠，说感武丁。 俊义密勿，多士实宁。 晋楚更霸，赵魏困横。 假途灭虢，践土会盟。 何遵约法，韩弊烦刑。 起翦颇牧，用军最精。 宣威沙漠，驰誉丹青。 九州禹迹，百郡秦并。 岳宗泰岱，禅主云亭。 雁门紫塞，鸡田赤诚。 昆池碣石，钜野洞庭。 旷远绵邈，岩岫杳冥。 治本于农，务兹稼穑。 俶载南亩，我艺黍稷。 税熟贡新，劝赏黜陟。 孟轲敦素，史鱼秉直。 庶几中庸，劳谦谨敕。 聆音察理，鉴貌辨色。 贻厥嘉猷，勉其祗植。 省躬讥诫，宠增抗极。 殆辱近耻，林皋幸即。 两疏见机，解组谁逼。 索居闲处，沉默寂寥。 求古寻论，散虑逍遥。 欣奏累遣，戚谢欢招。 渠荷的历，园莽抽条。 枇杷晚翠，梧桐蚤凋。 陈根委翳，落叶飘摇。 游鹍独运，凌摩绛霄。 耽读玩市，寓目囊箱。 易輶攸畏，属耳垣墙。 具膳餐饭，适口充肠。 饱饫烹宰，饥厌糟糠。 亲戚故旧，老少异粮。 妾御绩纺，侍巾帷房。 纨扇圆洁，银烛炜煌。 昼眠夕寐，蓝笋象床。 弦歌酒宴，接杯举殇。 矫手顿足，悦豫且康。 嫡后嗣续，祭祀烝尝。 稽颡再拜，悚惧恐惶。 笺牒简要，顾答审详。 骸垢想浴，执热愿凉。 驴骡犊特，骇跃超骧。 诛斩贼盗，捕获叛亡。 布射僚丸，嵇琴阮箫。 恬笔伦纸，钧巧任钓。 释纷利俗，并皆佳妙。 毛施淑姿，工颦妍笑。 年矢每催，曦晖朗曜。 璇玑悬斡，晦魄环照。 指薪修祜，永绥吉劭。 矩步引领，俯仰廊庙。 束带矜庄，徘徊瞻眺。 孤陋寡闻，愚蒙等诮。 谓语助者，焉哉乎也。"],
    ["前不久，家人因皮肤问题去工人医院看病，完了之后医生开了点药，并让搭配另一种洗护产品使用，医生指定让去医院外面的某某药房买，说医院没卖，其他药店也没卖。去了之后发现价格还不低，300左右，还不能刷医保。过了几天我在网上一搜，发现这套产品网上才卖200不到。同样的情况遇到过很多次，比如，我儿子满月时长了婴儿痤疮和湿疹，去儿童医院看，医生也是开了点药同时让去指定药店买另外几种药搭配着使用，同样不能刷医保，几次买下来花了近千块。上述两次都是皮肤科，但是其他科也存在同样的情况。我不能理解几个问题：为何经常用到得到的药，医院不采购进医院，以方便市民刷医保使用；为何指定药店才能买得到相关药品？是被院方合作的药店垄断了吗？然后医生开这些药后是有提成吗？最后，就算真要指定药店买，为何不能刷医保？老百姓看病本来就够难受的了，还是希望打针吃药时能省的就省下来。"],
]


demo = gr.Interface(
    fn=sentiment_analysis, 
    inputs=gr.Textbox(placeholder="输入文本...", lines=4), 
    outputs=gr.Label(label="keywords"),
    title=title,
    examples=examples)

demo.launch(share=True)